# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as they have the expertise in statistical modeling and machine learning.

Action: Delegate work to co-worker
Action Input: 
{
  "coworker": "Data Analyst",
  "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
  "context": "The task involves analyzing market data for AAPL to identify trends and predict market movements. Use statistical modeling and machine learning techniques to gain insights and predict future market movements."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and p

Final Answer: 
Based on the insights gathered from the Data Analyst Agent and internet search results, AAPL stock has shown positive trends recently. The stock has risen by 2.24% compared to the previous week, with a 16.08% increase over the last month and a 10.97% increase over the last year. The market cap of Apple Inc as of May 20, 2024, is $2911.48B.

Considering these trends and the risk tolerance level of the user for day trading preferences, it may be advantageous to develop trading strategies that capitalize on the recent positive momentum of AAPL stock. However, it is important to conduct further analysis and consider factors such as price targets, analyst predictions, and overall market conditions before finalizing any trading strategies.

> Finished chain.
 

Based on the insights gathered from the Data Analyst Agent and internet search results, AAPL stock has shown positive trends recently. The stock has risen by 2.24% compared to the previous week, with a 16.08% increase o

Final Answer: The recent positive momentum of AAPL stock can be attributed to various factors, including strong financial performance, market demand for their products, and overall market sentiment. To capitalize on this momentum for the Momentum Trading Strategy, it is essential to closely monitor key events, such as earnings reports, product launches, and market trends. Additionally, conducting thorough research and analysis on AAPL's valuation measures, profitability, and balance sheet can provide valuable insights for making informed trading decisions. It is crucial to stay updated on the latest news and developments related to AAPL to effectively execute the Momentum Trading Strategy and maximize returns.

> Finished chain.
 

The recent positive momentum of AAPL stock can be attributed to various factors, including strong financial performance, market demand for their products, and overall market sentiment. To capitalize on this momentum for the Momentum Trading Strategy, it is e

Final Answer: 
After conducting a thorough analysis of potential risks associated with the proposed trading strategies and execution plans for AAPL, it is crucial to consider various types of financial risks that could impact trading activities. These risks include systematic risks such as market risk, interest rate risk, and political risk, as well as unsystematic risks like operational risk, legal risk, and credit risk. Additionally, factors like country risk, foreign-exchange risk, and liquidity risk should be carefully assessed to ensure that trading decisions align with the firm's risk tolerance.

Moreover, understanding the basics of risk, the risk-return tradeoff, and the importance of diversification in managing risks is essential for maximizing returns and minimizing potential losses. By staying informed about the latest news and developments related to AAPL, continuously monitoring market conditions, setting clear entry and exit points based on the stock's upward trend, and a

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

Introduction:
This risk analysis report aims to identify and assess the potential risks associated with the proposed trading strategies and execution plans for AAPL. By analyzing various types of financial risks and understanding the importance of risk management strategies, this report will provide recommendations for mitigating risks and maximizing returns.

Key Risks:
1. Market Risk: The recent positive momentum of AAPL stock can be impacted by market fluctuations, economic conditions, and geopolitical events. It is crucial to closely monitor market trends and adjust trading strategies accordingly to mitigate market risk.

2. Interest Rate Risk: Changes in interest rates can affect the valuation of AAPL stock and impact trading decisions. By staying informed about interest rate movements and their potential impact on stock prices, traders can minimize interest rate risk.

3. Political Risk: Political events, regulatory changes, and government policies can influence the performance of AAPL stock. It is essential to assess political risk factors and consider their implications for trading activities.

4. Operational Risk: Operational failures, technology disruptions, and human errors can lead to financial losses and impact trading outcomes. Implementing robust operational risk management practices and contingency plans is crucial for mitigating operational risk.

5. Legal Risk: Legal disputes, regulatory compliance issues, and litigation can pose legal risks for AAPL stock trading. By ensuring compliance with laws and regulations and monitoring legal developments, traders can reduce legal risk exposure.

6. Credit Risk: Counterparty default, credit rating downgrades, and credit market disruptions can affect trading activities involving AAPL stock. Conducting credit risk assessments and diversifying counterparty exposure can help mitigate credit risk.

Mitigation Strategies:
1. Risk Management Strategies: Implement risk management practices to identify, assess, and monitor risks associated with trading activities for AAPL. Develop risk mitigation strategies and contingency plans to address potential risks proactively.

2. Diversification: Diversify trading portfolios and asset allocations to reduce concentration risk and enhance risk-adjusted returns. By diversifying investments across different asset classes and sectors, traders can minimize portfolio risk.

3. Continuous Monitoring: Continuously monitor market conditions, news events, and key developments related to AAPL to stay informed about potential risks and opportunities. Regularly assess trading strategies and adjust positions as needed to align with market trends.

4. Setting Clear Entry and Exit Points: Establish clear entry and exit points based on AAPL's upward trend, target price, and trading objectives. By setting predefined trading parameters and risk thresholds, traders can make informed decisions and manage risks effectively.

Conclusion:
By incorporating risk management strategies, diversifying trading portfolios, continuously monitoring market conditions, and setting clear entry and exit points, traders can optimize the execution of the proposed trading strategies and execution plans for AAPL. By proactively identifying and mitigating potential risks, traders can maximize returns and minimize losses while aligning trading activities with the firm's risk tolerance.